# Create Datasets

In [2]:
import os
import numpy as np
from PIL import Image

def divide_image_into_quadrants(image, n):
    
    if(len(image.shape) == 3):
        rows, cols, _ = image.shape  
    else:
        rows, cols = image.shape  
    
    num_rows = int(n ** 0.5)
    num_cols = n // num_rows
    
    quadrant_rows = rows // num_rows
    quadrant_cols = cols // num_cols
    
    quadrants = []
    
    for i in range(num_rows):
        for j in range(num_cols):
            start_row = i * quadrant_rows
            end_row = start_row + quadrant_rows
            start_col = j * quadrant_cols
            end_col = start_col + quadrant_cols
            
            quadrant = image[start_row:end_row, start_col:end_col]
            quadrants.append(quadrant)
    
    return quadrants

def process_8bit_images(path_8bit_source, path_8bit_destiny):
    
    for image_name in os.listdir(path_8bit_source):
        print(f"{path_8bit_source}{image_name}")
        image8bits = Image.open(f"{path_8bit_source}{image_name}").convert('CMYK')
        B = 4
        multi = np.zeros((B, image8bits.size[1], image8bits.size[0]))

        for i in range (0, B):
            multi[i,:,:] = np.array(image8bits.getchannel(i))


        
        image_raw = np.moveaxis(multi, 0, -1)

        quadrants = divide_image_into_quadrants(image_raw, 4)


        for idx,q in enumerate(quadrants):  
            print(f"Saving image {image_name[:-4]}_{idx} with shape {q.shape}")
            newFile = open(f"{path_8bit_destiny}{os.path.splitext(image_name)[0]}_{idx}.rawb", "wb")
            q.tofile(newFile, format="%d")
            newFile.close()



In [ ]:
# TRAIN
path_8bit_train = "" #Path to the 8bit images
path_8bit_train_destiny = "" #Path to save the 8bit images in quadrants
process_8bit_images(path_8bit_train, path_8bit_train_destiny)

In [ ]:
# TEST
path_8bit_test = ""#Path to the 8bit images
path_8bit_test_destiny = ""#Path to save the 8bit images in quadrants
process_8bit_images(path_8bit_test, path_8bit_test_destiny)

# Read function

In [ ]:
def read_rawb_NirRGB(file_name):
    SMALL_IMAGE_SHAPE = (3454, 3650, 4)
    with open(file_name, 'rb') as file:
        img = file.read()
        
    img = np.frombuffer(img, dtype=np.float64)

    img = img.reshape(SMALL_IMAGE_SHAPE)

    return img.astype(np.uint8)

def read_rawb_RGB(file_name):
    img = read_rawb_NirRGB(file_name)
    
    return img[:, :, 1:4]  # Delete Nir channel